loading datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step5_ws')

In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=972396ba2d1327fca7693e8dc3e930fa1b5b11d481e94dd3245e7443e22f10f4
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=f4b84bb139b2664ef28b81d1468837f5fca79c6614db599bf778579dcdd85906
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


find checkpoint function

In [ ]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    # Extract epoch number from filename
    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    # Sort by epoch descending
    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest


In [ ]:
import numpy as np
from collections import defaultdict
from torch.utils.data import WeightedRandomSampler


def compute_class_frequencies(dataset, num_classes=19):
    class_counts = np.zeros(num_classes)
    for _, mask in dataset:
        mask_np = mask.squeeze().numpy() if torch.is_tensor(mask) else np.array(mask)
        valid_mask = mask_np != 255  # ignore label 255
        for cls in np.unique(mask_np[valid_mask]):
            class_counts[cls] += np.sum(mask_np[valid_mask] == cls)
    return class_counts / class_counts.sum()

def get_image_class_map(dataset, num_classes=19):
    image_class_map = []
    for idx in range(len(dataset)):
        _, mask = dataset[idx]
        mask_np = mask.squeeze().numpy() if torch.is_tensor(mask) else np.array(mask)
        valid_mask = mask_np != 255
        present_classes = np.unique(mask_np[valid_mask])
        image_class_map.append((idx, present_classes))
    return image_class_map


def build_weighted_sampler(image_class_map, class_frequencies, num_samples):
    class_weights = 1.0 / (class_frequencies + 1e-6)  # avoid division by zero
    image_weights = []

    for _, classes in image_class_map:
        weights = [class_weights[c] for c in classes]
        image_weights.append(float(np.max(weights)))  # prioritize rarest class in image

    sampler = WeightedRandomSampler(weights=image_weights, num_samples=num_samples, replacement=True)
    return sampler



main

In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5 import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch

def main():
    num_classes = 19
    transform = transforms.Compose([
        transforms.Resize((720, 1280)),
        transforms.ToTensor()
    ])

    target_transform = transforms.Compose([
        transforms.Resize((720, 1280), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

    dataset_root = '/content/dataset/GTA5'

    full_dataset = GTA5(root=dataset_root, transform=transform, target_transform=target_transform)
    print(f"Full dataset size: {len(full_dataset)}")

    # Genera gli indici
    indices = list(range(len(full_dataset)))

    # Split in train/test (33% test set come nel tuo esempio)
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)

    # Crea i subset
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Val dataset size: {len(val_dataset)}")

    print(f"Train dataset size: {len(train_dataset)}")

    print("Computing class frequencies and sampler...")
    class_freqs = compute_class_frequencies(train_dataset, num_classes=num_classes)
    image_class_map = get_image_class_map(train_dataset, num_classes=num_classes)
    sampler = build_weighted_sampler(image_class_map, class_freqs, num_samples=len(train_dataset))

    train_loader = DataLoader(train_dataset, batch_size=2, sampler=sampler, num_workers=2)



    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive/MLDL_repo/step5_ws/check_ws"

    if not os.path.exists(checkpoint_dir):
      os.makedirs(checkpoint_dir)

    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0  # Non serve più riprendere dal batch
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")


    # Check Data and Labels
    for images, labels in train_loader:
        print("Images shape:", images.shape, "dtype:", images.dtype)
        print("Labels shape:", labels.shape, "dtype:", labels.dtype)
        print("Unique labels:", torch.unique(labels))
        break  # Print for the first batch only

    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)


    final_model_path = "/content/drive/MyDrive/MLDL_repo/step5_ws/check_ws/final_ws.pt"
    torch.save(model.state_dict(), final_model_path)
    print(f" Final model saved in: {final_model_path}")

if __name__ == "__main__":
    main()


Full dataset size: 2500
Train dataset size: 1750
Val dataset size: 750
Train dataset size: 1750
Computing class frequencies and sampler...


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 228MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 228MB/s]


Output streaming troncato alle ultime 5000 righe.
📉 Loss: 0.07591705769300461

 Epoch 49/50 - Batch 87/875
📉 Loss: 0.0839647725224495

 Epoch 49/50 - Batch 88/875
📉 Loss: 0.09877175837755203

 Epoch 49/50 - Batch 89/875
📉 Loss: 0.07142047584056854

 Epoch 49/50 - Batch 90/875
📉 Loss: 0.04474753886461258

 Epoch 49/50 - Batch 91/875
📉 Loss: 0.07448480278253555

 Epoch 49/50 - Batch 92/875
📉 Loss: 0.07412221282720566

 Epoch 49/50 - Batch 93/875
📉 Loss: 0.09834615141153336

 Epoch 49/50 - Batch 94/875
📉 Loss: 0.07112865895032883

 Epoch 49/50 - Batch 95/875
📉 Loss: 0.05387699976563454

 Epoch 49/50 - Batch 96/875
📉 Loss: 0.06657374650239944

 Epoch 49/50 - Batch 97/875
📉 Loss: 0.09318932145833969

 Epoch 49/50 - Batch 98/875
📉 Loss: 0.06170928105711937

 Epoch 49/50 - Batch 99/875
📉 Loss: 0.05685969442129135

 Epoch 49/50 - Batch 100/875
📉 Loss: 0.07815834134817123

 Epoch 49/50 - Batch 101/875
📉 Loss: 0.07113748043775558

 Epoch 49/50 - Batch 102/875
📉 Loss: 0.0837814137339592

 Epoch 4

validation

In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate


#initialize model
num_classes = 19
context_path = 'resnet18'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiSeNet(num_classes=num_classes, context_path=context_path).to(device)


#load model
model_path = "/content/drive/MyDrive/MLDL_repo/step5_ws/check_ws/final_ws.pt"
model.load_state_dict(torch.load(model_path, map_location='cpu'), strict = False )

model.eval() #python built in function

# Dataset definitions and transformation
transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
print(f"Test dataset size: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)


#results
best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)
print("\n Risultati su Cityscape Validation:")
print(f" - mIoU: {miou:.4f}")
for idx, iou in enumerate(per_class_ious):
        print(f"Classe {idx}: IoU = {iou:.4f}")



Loaded 500 images for split: val
Test dataset size: 500


🔍 Validating: 100%|██████████| 250/250 [00:29<00:00,  8.57it/s]

 Validation mIoU: 0.2038
 New best mIoU found!

 Risultati su Cityscape Validation:
 - mIoU: 0.2038
Classe 0: IoU = 0.5653
Classe 1: IoU = 0.0669
Classe 2: IoU = 0.6633
Classe 3: IoU = 0.0581
Classe 4: IoU = 0.0567
Classe 5: IoU = 0.0009
Classe 6: IoU = 0.0113
Classe 7: IoU = 0.0039
Classe 8: IoU = 0.7409
Classe 9: IoU = 0.0562
Classe 10: IoU = 0.6986
Classe 11: IoU = 0.2597
Classe 12: IoU = 0.0194
Classe 13: IoU = 0.5470
Classe 14: IoU = 0.0294
Classe 15: IoU = 0.0087
Classe 16: IoU = 0.0100
Classe 17: IoU = 0.0648
Classe 18: IoU = 0.0109
